# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")


In [2]:
from datetime import datetime,timedelta
date_time = datetime.now() - timedelta(1)
date_time = date_time.strftime("%Y/%m/%d")
print("取得昨日時間:",date_time)


取得昨日時間: 2020/01/03


In [3]:
# 利用 selenium 持續執行下滑的動作，直到取得「昨天時間」的新聞，表示今天的已經取得完畢
import time
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(2)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html.parser")
    print(soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text)
    if date_time in soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text:
        print('==== STOP ====')
        break

2020/01/03 22:51
==== STOP ====


In [4]:
# 取得資料
html_source = browser.page_source
soup = BeautifulSoup(html_source, "html.parser")

for d in soup.find(class_="part_list_2").find_all('h3'):
    if date_time in d.find(class_="date").text :
        break
    print(d.find(class_="date").text, d.find_all('a')[-1].text)

2020/01/04 10:26 無法參加韓國瑜洪秀柱聯合造勢　「沒接到電話」王金平這麼說
2020/01/04 10:23 跟陳偉殷被馬林魚同月拋棄　卡斯楚帶槍投靠衛冕軍國民
2020/01/04 10:22 工廠男手遭機器捲入截肢　車禍腦死男捐出手臂「異體縫合」
2020/01/04 10:17 2港女確診染「甲型流感」...港府啟動應變措施！專家：疫情會慢慢好轉
2020/01/04 10:15 最大獎30萬元！電子代工五哥尾牙拚場　千萬總獎金犒賞員工
2020/01/04 10:15 「不同立場還是可以團結」　林鶴明：蔡英文的責任是維護國家主權
2020/01/04 10:15 沈一鳴禮賢下士！軍事專家以周瑜盛讚：如飲醇醪不覺自醉
2020/01/04 10:13 搶不到票看過來！台鐵春節再加開8班東線列車　1/7凌晨開賣
2020/01/04 10:12 擊殺伊朗「靈魂人物」蘇萊曼尼　川普電視演說：是為了停止戰爭！
2020/01/04 10:08 蔡英文台南競總5日辦大型造勢　一分局交管疏導
2020/01/04 10:05 台北車展／展期最後倒數　這6款性能靚車你有拍照留念嗎？
2020/01/04 10:04 香港懷疑「武漢肺炎」個案7宗　港府啟動「嚴重應變級別」
2020/01/04 10:02 全球每20秒賣一瓶！Kiehl’s「神奇化妝水」搞定所有肌膚問題
2020/01/04 10:02 訪港遊客人數大跌！海洋公園宣佈「2020年員工凍薪」：鼓勵員工提早退休
2020/01/04 10:01 遺書留「OO我愛妳…」！恐怖情人奪3命跳樓跪姿亡　女友裝死逃過血腥屠殺
2020/01/04 10:00 【毛孩小學堂】廁所髒了不想上？貓主子抗議貓砂盆太髒的4行為！
2020/01/04 10:00 台灣燈會宣導森林保育！邀俄籍畫家卡佳打造「守護者石虎」燈座
2020/01/04 10:00 哺乳媽媽接種流感疫苗不只保護自己　抗體可從母乳傳給寶寶
2020/01/04 10:00 韓式炸雞堆超高！台中浮誇系丼飯　半隻烤雞鋪飯上3百有找
2020/01/04 09:57 4導彈炸翻車隊…伊朗少將遭狙殺畫面曝光！僅剩左手「紅寶石戒」認身分
2020/01/04 09:52 網拍色狼「偽催眠術」迷昏性侵少女　錄私密影片再逼床戰：每天洗澡視訊
2020/01/

In [5]:
# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    if date_time in d.find(class_="date").text :
        break
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all('a')[-1].text
    })

import pandas as pd
pd.DataFrame(news)

,時間,標題
0,2020/01/04 10:26,無法參加韓國瑜洪秀柱聯合造勢 「沒接到電話」王金平這麼說
1,2020/01/04 10:23,跟陳偉殷被馬林魚同月拋棄 卡斯楚帶槍投靠衛冕軍國民
2,2020/01/04 10:22,工廠男手遭機器捲入截肢 車禍腦死男捐出手臂「異體縫合」
3,2020/01/04 10:17,2港女確診染「甲型流感」...港府啟動應變措施！專家：疫情會慢慢好轉
4,2020/01/04 10:15,最大獎30萬元！電子代工五哥尾牙拚場 千萬總獎金犒賞員工
5,2020/01/04 10:15,「不同立場還是可以團結」 林鶴明：蔡英文的責任是維護國家主權
6,2020/01/04 10:15,沈一鳴禮賢下士！軍事專家以周瑜盛讚：如飲醇醪不覺自醉
7,2020/01/04 10:13,搶不到票看過來！台鐵春節再加開8班東線列車 1/7凌晨開賣
8,2020/01/04 10:12,擊殺伊朗「靈魂人物」蘇萊曼尼 川普電視演說：是為了停止戰爭！
9,2020/01/04 10:08,蔡英文台南競總5日辦大型造勢 一分局交管疏導


In [6]:
# 關閉瀏覽器

browser.close()

## 2. 取出現在時間兩小時內的新聞

In [7]:
# 取得兩小時內的時間
from datetime import datetime, timedelta
two_hours_ago_time = datetime.now() - timedelta(hours=2)

# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < two_hours_ago_time:
        break
    
    news.append({
        '時間': d.find(class_="date").text,
        '標題': d.find_all('a')[-1].text
    })

import pandas as pd
pd.DataFrame(news)

,時間,標題
0,2020/01/04 10:26,無法參加韓國瑜洪秀柱聯合造勢 「沒接到電話」王金平這麼說
1,2020/01/04 10:23,跟陳偉殷被馬林魚同月拋棄 卡斯楚帶槍投靠衛冕軍國民
2,2020/01/04 10:22,工廠男手遭機器捲入截肢 車禍腦死男捐出手臂「異體縫合」
3,2020/01/04 10:17,2港女確診染「甲型流感」...港府啟動應變措施！專家：疫情會慢慢好轉
4,2020/01/04 10:15,最大獎30萬元！電子代工五哥尾牙拚場 千萬總獎金犒賞員工
5,2020/01/04 10:15,「不同立場還是可以團結」 林鶴明：蔡英文的責任是維護國家主權
6,2020/01/04 10:15,沈一鳴禮賢下士！軍事專家以周瑜盛讚：如飲醇醪不覺自醉
7,2020/01/04 10:13,搶不到票看過來！台鐵春節再加開8班東線列車 1/7凌晨開賣
8,2020/01/04 10:12,擊殺伊朗「靈魂人物」蘇萊曼尼 川普電視演說：是為了停止戰爭！
9,2020/01/04 10:08,蔡英文台南競總5日辦大型造勢 一分局交管疏導


## 3. 根據範例，取出三天前下午三點到五點的新聞

In [8]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("https://www.ettoday.net/news/news-list.htm")

In [9]:
# 取得三天前下午三點到五點的時間
from datetime import datetime, timedelta

date_time = datetime.now() - timedelta(3)
print("取得昨天時間：", date_time)

date_time_month = date_time.strftime("%m")
date_time_day = date_time.strftime("%d")
print("取得昨天時間（月）：", date_time_month)
print("取得昨天時間（日）：", date_time_day)


three_days_15_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 15', '%Y/%m/%d %H')
three_days_17_clock = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 17', '%Y/%m/%d %H')
print("取得昨天下午三點：", three_days_15_clock)
print("取得昨天下午五點：", three_days_17_clock)

取得昨天時間： 2020-01-01 10:29:05.881826
取得昨天時間（月）： 01
取得昨天時間（日）： 01
取得昨天下午三點： 2020-01-01 15:00:00
取得昨天下午五點： 2020-01-01 17:00:00


In [10]:
# 選擇且點選日期
from selenium.webdriver.support.ui import Select

selM = Select(browser.find_element_by_id("selM"))
selM.select_by_value(date_time_month)
selD = Select(browser.find_element_by_id("selD"))
selD.select_by_value(date_time_day)
browser.find_element_by_id('button').click()

NoSuchElementException: Message: Cannot locate option with value: 01


In [11]:
# 利用 selenium 持續執行下滑的動作，直到取得要求時間內的新聞，表示取得完畢
import time
while True:
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(1)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, "html.parser")
    new_date = soup.find_all(class_="part_list_2")[-1].find_all('h3')[-1].find(class_="date").text
    print(new_date)
    if datetime.strptime(new_date, '%Y/%m/%d %H:%M') < three_days_15_clock:
        print('==== STOP ====')
        break

2020/01/03 23:07
2020/01/03 22:09
2020/01/03 21:39
2020/01/03 21:20
2020/01/03 21:03
2020/01/03 20:47
2020/01/03 20:25
2020/01/03 20:08
2020/01/03 19:36
2020/01/03 19:27
2020/01/03 19:04
2020/01/03 18:48
2020/01/03 18:34
2020/01/03 18:23
2020/01/03 18:15
2020/01/03 18:02
2020/01/03 17:55
2020/01/03 17:50
2020/01/03 17:40
2020/01/03 17:35
2020/01/03 17:28
2020/01/03 17:21
2020/01/03 17:08
2020/01/03 17:05
2020/01/03 17:00
2020/01/03 16:50
2020/01/03 16:37
2020/01/03 16:26
2020/01/03 16:18
2020/01/03 16:10
2020/01/03 16:06
2020/01/03 16:01
2020/01/03 15:50
2020/01/03 15:45
2020/01/03 15:38
2020/01/03 15:31
2020/01/03 15:25
2020/01/03 15:15
2020/01/03 15:03
2020/01/03 15:00
2020/01/03 14:51
2020/01/03 14:44
2020/01/03 14:26
2020/01/03 14:10
2020/01/03 13:54
2020/01/03 13:30
2020/01/03 13:16
2020/01/03 13:03
2020/01/03 12:51
2020/01/03 12:41
2020/01/03 12:28
2020/01/03 12:19
2020/01/03 12:06
2020/01/03 12:00
2020/01/03 11:53
2020/01/03 11:45
2020/01/03 11:35
2020/01/03 11:23
2020/01/03 11:

KeyboardInterrupt: 

In [ ]:
# 取得資料（整理成好的格式）
news = []

for d in soup.find(class_="part_list_2").find_all('h3'):
    # 比兩小時前早的話就結束
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < three_days_15_clock:
        print('==== STOP ====')
        break
    
    if datetime.strptime(d.find(class_="date").text, '%Y/%m/%d %H:%M') < three_days_17_clock:
        news.append({
            '時間': d.find(class_="date").text,
            '標題': d.find_all('a')[-1].text
        })

import pandas as pd
pd.DataFrame(news)

In [ ]:
# 關閉瀏覽器

browser.close()